In [ ]:
import pandas as pd
import numpy as np

def gaussian_wave(t, peak_time, amplitude, width):
    """Generate Gaussian-shaped ERP component"""
    return amplitude * np.exp(-0.5 * ((t - peak_time) / width) ** 2)

def generate_erp_waveform(duration=2, sampling_rate=250, save_path='data\\artificial_erp.csv'):
    """Generate artificial ERP waveforms with P300, N100, and other components"""
    
    # Time vector (in seconds)
    t = np.linspace(0, duration, int(duration * sampling_rate))
    
    # EEG channels - focus on central/parietal where ERPs are prominent
    channels = ['Fz', 'Cz', 'Pz', 'C3', 'C4', 'P3', 'P4', 'Oz']
    
    data = {}
    data['Time'] = t * 1000  # Convert to milliseconds for easier interpretation
    
    for i, ch in enumerate(channels):
        # Initialize baseline
        signal = np.zeros(len(t))
        
        # ERP Components (amplitudes in microvolts)
        if ch in ['Fz', 'Cz']:  # Frontal/Central - stronger N100, P200
            signal += gaussian_wave(t, 0.100, -5, 0.030)   # N100 (100ms, -5μV)
            signal += gaussian_wave(t, 0.200, 8, 0.050)    # P200 (200ms, +8μV)
            signal += gaussian_wave(t, 0.300, 12, 0.080)   # P300 (300ms, +12μV)
            
        elif ch in ['Pz', 'P3', 'P4']:  # Parietal - strongest P300
            signal += gaussian_wave(t, 0.080, -3, 0.025)   # N80 (80ms, -3μV)
            signal += gaussian_wave(t, 0.100, -4, 0.030)   # N100 (100ms, -4μV)
            signal += gaussian_wave(t, 0.300, 15, 0.100)   # P300 (300ms, +15μV) - strongest
            signal += gaussian_wave(t, 0.600, 5, 0.120)    # Late Positive Component
            
        elif ch == 'Oz':  # Occipital - visual ERPs
            signal += gaussian_wave(t, 0.075, 6, 0.020)    # P75 (75ms, +6μV)
            signal += gaussian_wave(t, 0.100, -8, 0.025)   # N100 (100ms, -8μV)
            signal += gaussian_wave(t, 0.170, 10, 0.040)   # P170 (170ms, +10μV)
            
        else:  # Other channels - moderate responses
            signal += gaussian_wave(t, 0.100, -3, 0.030)   # N100
            signal += gaussian_wave(t, 0.300, 8, 0.080)    # P300
        
        # Add some realistic noise and baseline drift
        noise = 0.5 * np.random.randn(len(t))
        baseline_drift = 0.1 * np.sin(2 * np.pi * 0.5 * t)  # Slow drift
        
        # Combine all components
        final_signal = signal + noise + baseline_drift
        
        # Convert to microvolts (same as your unit_conversion)
        data[ch] = final_signal * 1e-6
    
    # Add Status column (0 for baseline, 1 for stimulus onset at t=0)
    status = np.zeros(len(t))
    status[0] = 1  # Stimulus marker at beginning
    data['Status'] = status
    
    # Create DataFrame and save
    df = pd.DataFrame(data)
    df.to_csv(save_path, index=False)
    print(f"Artificial ERP data saved to: {save_path}")
    print(f"Duration: {duration}s, Sampling rate: {sampling_rate}Hz")
    print("ERP Components included:")
    print("- N100: Negative deflection ~100ms")
    print("- P200: Positive deflection ~200ms") 
    print("- P300: Large positive deflection ~300ms")
    print("- Late components up to 600ms")
    return save_path

# Generate the ERP data
path = generate_erp_waveform()
print(f"\nUse this path in your code: path = '{path}'")